<a href="https://colab.research.google.com/github/sf-rahul/Tensorflow/blob/master/charRNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
lst = [[1,2,3],[5,6,7]]


In [0]:
import numpy as np
lst = np.array(lst)

In [15]:
lst[:,-1:]

array([[3],
       [7]])